In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV as RSC
from sklearn.model_selection import *
from sklearn.ensemble import GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier

In [2]:
df=pd.read_csv("hr_train.csv")
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.42,0.46,2,150,3,0,1,0,sales,medium
1,0.66,0.77,2,171,2,0,0,0,technical,medium
2,0.55,0.49,5,240,3,0,0,0,technical,high
3,0.22,0.88,4,213,3,1,0,0,technical,medium
4,0.20,0.72,6,224,4,0,1,0,technical,medium


In [3]:
df.left.value_counts()

0    7424
1    3075
Name: left, dtype: int64

In [4]:
df.shape

(10499, 10)

In [5]:
cat_cols=df.select_dtypes(['object']).columns
cat_cols

Index(['sales', 'salary'], dtype='object')

In [6]:
df[cat_cols[0]].value_counts(),df[cat_cols[1]].value_counts()


(sales          2935
 technical      1890
 support        1556
 IT              825
 product_mng     639
 marketing       614
 hr              532
 RandD           530
 accounting      527
 management      451
 Name: sales, dtype: int64,
 low       5104
 medium    4515
 high       880
 Name: salary, dtype: int64)

In [7]:
for c in cat_cols:
    f=df[c].value_counts()
    selected_cats=f.index[f>500][:-1]
    print(c)
    for cat in selected_cats:
        n=c+'_'+cat
        df[n]=(df[c]==cat).astype(int)
    del df[c]

sales
salary


In [8]:
df.columns


Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident', 'left',
       'promotion_last_5years', 'sales_sales', 'sales_technical',
       'sales_support', 'sales_IT', 'sales_product_mng', 'sales_marketing',
       'sales_hr', 'sales_RandD', 'salary_low', 'salary_medium'],
      dtype='object')

In [9]:
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_sales,sales_technical,sales_support,sales_IT,sales_product_mng,sales_marketing,sales_hr,sales_RandD,salary_low,salary_medium
0,0.42,0.46,2,150,3,0,1,0,1,0,0,0,0,0,0,0,0,1
1,0.66,0.77,2,171,2,0,0,0,0,1,0,0,0,0,0,0,0,1
2,0.55,0.49,5,240,3,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0.22,0.88,4,213,3,1,0,0,0,1,0,0,0,0,0,0,0,1
4,0.20,0.72,6,224,4,0,1,0,0,1,0,0,0,0,0,0,0,1


In [10]:
def report(results,n_top=3):
    for i in range(1,n_top+1):
        c=np.flatnonzero(results['rank_test_score']==i)
        for ci in c:
            print("Rank:",i)
            print("Mean Validation Score:",results['mean_test_score'][ci],results['std_test_score'][ci])
            print('Parameters:',results['params'][ci])
            print("")
            

In [11]:
xgb_params = {  
                "learning_rate":[0.01,0.05,0.1,0.3,0.5],
                "gamma":[i/10.0 for i in range(0,5)],
                "max_depth": [2,3,4,5,6,7,8],
                "min_child_weight":[1,2,5,10],
                "max_delta_step":[0,1,2,5,10],
                "subsample":[i/10.0 for i in range(5,10)],
                "colsample_bytree":[i/10.0 for i in range(5,10)],
                "colsample_bylevel":[i/10.0 for i in range(5,10)],
                "reg_lambda":[1e-5, 1e-2, 0.1, 1, 100], 
                "reg_alpha":[1e-5, 1e-2, 0.1, 1, 100],
                "scale_pos_weight":[1,2,3,4,5,6,7,8,9],
                "n_estimators":[100,500,700,1000]
             }

In [12]:
xgb=XGBClassifier(objective='binary:logistic')

In [13]:
n_iter=10
rs=RSC(xgb,n_jobs=-1,cv=5,n_iter=n_iter,scoring='roc_auc',
                 param_distributions=xgb_params)

In [14]:
x_train=df.drop('left',axis=1)
y_train=df.left
x_train.columns

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'sales_sales', 'sales_technical',
       'sales_support', 'sales_IT', 'sales_product_mng', 'sales_marketing',
       'sales_hr', 'sales_RandD', 'salary_low', 'salary_medium'],
      dtype='object')

In [15]:
rs.fit(x_train,y_train)

C:\Users\menta\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:59:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.3,
                         

In [16]:
report(rs.cv_results_,5)

Rank: 1
Mean Validation Score: 0.8366466799493336 0.019490666758777014
Parameters: {'subsample': 0.6, 'scale_pos_weight': 3, 'reg_lambda': 0.1, 'reg_alpha': 100, 'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 7, 'max_delta_step': 10, 'learning_rate': 0.01, 'gamma': 0.3, 'colsample_bytree': 0.6, 'colsample_bylevel': 0.5}

Rank: 2
Mean Validation Score: 0.8357078835159829 0.018598479425509774
Parameters: {'subsample': 0.8, 'scale_pos_weight': 6, 'reg_lambda': 100, 'reg_alpha': 0.1, 'n_estimators': 700, 'min_child_weight': 1, 'max_depth': 8, 'max_delta_step': 0, 'learning_rate': 0.05, 'gamma': 0.2, 'colsample_bytree': 0.6, 'colsample_bylevel': 0.7}

Rank: 3
Mean Validation Score: 0.8354604758754167 0.01729408999695364
Parameters: {'subsample': 0.8, 'scale_pos_weight': 4, 'reg_lambda': 0.1, 'reg_alpha': 1, 'n_estimators': 1000, 'min_child_weight': 1, 'max_depth': 3, 'max_delta_step': 5, 'learning_rate': 0.1, 'gamma': 0.3, 'colsample_bytree': 0.5, 'colsample_bylevel': 0.5}

Rank: 

In [17]:
rs.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0.3, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=10, max_depth=7,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=100, reg_lambda=0.1, scale_pos_weight=3, subsample=0.6,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [18]:
xgb_params={
    'n_estimators':[100,500,700,900,1000,1200,1500]
}

In [19]:
xgb1=XGBClassifier(learning_rate=0.1,subsample=0.8,colsample_bylevel=0.8,colsample_bytree=0.8)

In [20]:
gs=GridSearchCV(xgb1,cv=5,param_grid=xgb_params,scoring='roc_auc',verbose=2,n_jobs=1)

In [21]:
gs.fit(x_train,y_train)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
[CV] n_estimators=100 ................................................
[17:59:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\menta\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV] ................................. n_estimators=100, total=   1.0s
[CV] n_estimators=100 ................................................
[17:59:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s
C:\Users\menta\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV] ................................. n_estimators=100, total=   0.9s
[CV] n_estimators=100 ................................................
[17:59:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[CV] ................................ n_estimators=1500, total=  13.2s
[CV] n_estimators=1500 ...............................................
[18:03:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\menta\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV] ................................ n_estimators=1500, total=  11.7s
[18:03:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Done  35 out of  35 | elapsed:  4.2min finished
C:\Users\menta\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=0.8,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=0.1, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=0.8,
                                     tree_method=None, val

In [22]:
report(gs.cv_results_,3)

Rank: 1
Mean Validation Score: 0.8395761985850957 0.016008427266475927
Parameters: {'n_estimators': 100}

Rank: 2
Mean Validation Score: 0.8368405083862976 0.017349085158461675
Parameters: {'n_estimators': 500}

Rank: 3
Mean Validation Score: 0.8354727851787216 0.01729864821635559
Parameters: {'n_estimators': 700}



In [23]:
gs.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.8,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [24]:
xgb_params={
    'gamma':[5,8,10,12,15],
    'max_depth':[6,7,8,9,10,11,12]
}

In [25]:
xgb2=XGBClassifier(learning_rate=0.1,n_estimators=100,subsample=0.8,min_child_weight=2,colsample_bylevel=0.8,colsample_bytree=0.8)

In [26]:
rs2=RSC(xgb2,param_distributions=xgb_params,n_iter=200,cv=5,scoring='roc_auc',n_jobs=1,verbose=2)

In [27]:
rs2.fit(x_train,y_train)

C:\Users\menta\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:278: UserWarning: The total space of parameters 35 is smaller than n_iter=200. Running 35 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\menta\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Fitting 5 folds for each of 35 candidates, totalling 175 fits
[CV] max_depth=6, gamma=5 ............................................
[18:03:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV] ............................. max_depth=6, gamma=5, total=   0.9s
[CV] max_depth=6, gamma=5 ............................................
[18:03:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
C:\Users\menta\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV] ............................. max_depth=6, gamma=5, total=   0.9s
[CV] max_depth=6, gamma=5 ............................................
[18:03:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\menta\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[CV] ........................... max_depth=12, gamma=15, total=   1.8s
[CV] max_depth=12, gamma=15 ..........................................
[18:07:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\menta\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV] ........................... max_depth=12, gamma=15, total=   1.8s
[18:07:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Done 175 out of 175 | elapsed:  3.9min finished
C:\Users\menta\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=0.8,
                                           colsample_bynode=None,
                                           colsample_bytree=0.8, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=0.1,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=2, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, reg_alpha=None,
                                

In [28]:
report(rs2.cv_results_,3)

Rank: 1
Mean Validation Score: 0.8417446993473992 0.017751952507613588
Parameters: {'max_depth': 12, 'gamma': 8}

Rank: 2
Mean Validation Score: 0.8416901203652237 0.018694776594418705
Parameters: {'max_depth': 12, 'gamma': 12}

Rank: 3
Mean Validation Score: 0.8416030518259389 0.01716596694218978
Parameters: {'max_depth': 6, 'gamma': 5}



In [29]:
rs2.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.8,
              colsample_bynode=1, colsample_bytree=0.8, gamma=8, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=12,
              min_child_weight=2, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [30]:
xgb_params={
            'max_delta_step':[0,1,3,6,10],
            'scale_pos_weight':[1,2,3,4]
            }

In [31]:
xgb3=XGBClassifier(learning_rate=0.1,n_estimators=100,min_child_weight=2,gamma=12,max_depth=8,
                  subsample=0.8,colsample_bylevel=0.8,colsample_bytree=0.8)

In [32]:
gs3=GridSearchCV(xgb3,param_grid=xgb_params,cv=5,scoring='roc_auc',n_jobs=-1,verbose=10)

In [33]:
gs3.fit(x_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   19.4s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   28.3s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   34.9s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   44.1s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   51.0s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.3min finished
C:\Users\menta\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option

[18:08:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=0.8,
                                     colsample_bynode=None,
                                     colsample_bytree=0.8, gamma=12,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=0.1, max_delta_step=None,
                                     max_depth=8, min_child_weight=2,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=0.8,
                                     tree_method=None, validate_pa

In [34]:
report(gs.cv_results_,3)

Rank: 1
Mean Validation Score: 0.8395761985850957 0.016008427266475927
Parameters: {'n_estimators': 100}

Rank: 2
Mean Validation Score: 0.8368405083862976 0.017349085158461675
Parameters: {'n_estimators': 500}

Rank: 3
Mean Validation Score: 0.8354727851787216 0.01729864821635559
Parameters: {'n_estimators': 700}



In [35]:
xgb_params={
            'subsample':[i/10 for i in range(5,11)],
            'colsample_bytree':[i/10 for i in range(5,11)],
            'colsample_bylevel':[i/10 for i in range(5,11)]
            }

In [36]:
xgb4=XGBClassifier(learning_rate=0.1,n_estimators=100,min_child_weight=2,gamma=12,max_depth=8,
                   scale_pos_weight=1,max_delta_step=0
                  )

In [37]:
rs4=RandomizedSearchCV(xgb4,param_distributions=xgb_params,cv=5,n_iter=20,scoring='roc_auc',
                                n_jobs=-1,verbose=20)

In [38]:
rs4.fit(x_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    8.3s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:   11.6s
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed:   

[18:10:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=12,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=8, min_child_weight=2,
                                           missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=None, reg_alpha=None,
                                      

In [39]:
report(rs4.cv_results_,3)

Rank: 1
Mean Validation Score: 0.8404334037900535 0.019698283279177782
Parameters: {'subsample': 0.8, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.6}

Rank: 2
Mean Validation Score: 0.8402982228806742 0.019473451861420257
Parameters: {'subsample': 0.8, 'colsample_bytree': 0.9, 'colsample_bylevel': 0.8}

Rank: 3
Mean Validation Score: 0.8401931728626006 0.019697473598526756
Parameters: {'subsample': 0.7, 'colsample_bytree': 0.8, 'colsample_bylevel': 0.9}



In [40]:
xgb5=XGBClassifier(learning_rate=0.1,n_estimators=100,min_child_weight=2,gamma=12,max_depth=8,
                   scale_pos_weight=1,max_delta_step=0,
                   colsample_bylevel= 1.0, colsample_bytree= 0.8, subsample= 0.7
                  )

In [41]:
xgb_params={
            'reg_lambda':[i/10 for i in range(0,50)],
            'reg_alpha':[i/10 for i in range(0,50)]
            }

In [42]:
rs4=RSC(xgb5,param_distributions=xgb_params,cv=5,n_iter=20,scoring='roc_auc',n_jobs=-1,verbose=10)

In [43]:
rs5=rs4
rs5.fit(x_train,y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   21.5s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   31.5s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   38.2s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   48.9s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:   56.1s
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.5min finished
C:\Users\menta\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option

[18:11:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=1.0,
                                           colsample_bynode=None,
                                           colsample_bytree=0.8, gamma=12,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=8, min_child_weight=2,
                                           missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           nu...
                                           verbosity=None),
                   n_iter=20, n_jobs=-1,
                   param_distributi

In [44]:
report(rs5.cv_results_,3)

Rank: 1
Mean Validation Score: 0.8390797957588877 0.01928507725581077
Parameters: {'reg_lambda': 4.0, 'reg_alpha': 1.9}

Rank: 2
Mean Validation Score: 0.8386988440419948 0.020076672990456474
Parameters: {'reg_lambda': 3.2, 'reg_alpha': 0.6}

Rank: 3
Mean Validation Score: 0.8386977950492293 0.020302542449606175
Parameters: {'reg_lambda': 2.4, 'reg_alpha': 1.2}



In [45]:
xgb6=XGBClassifier(learning_rate=0.1,n_estimators=100,min_child_weight=2,gamma=12,max_depth=8,
                  scale_pos_weight=1,max_delta_step=0,colsample_bylevel=1.0,colsample_bytree=0.8,
                  subsample=0.7,reg_lambda=1.8,reg_alpha=0)

In [46]:
xgb6.fit(x_train,y_train)

[18:11:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1.0,
              colsample_bynode=1, colsample_bytree=0.8, gamma=12, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=8,
              min_child_weight=2, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1.8, scale_pos_weight=1, subsample=0.7,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [47]:
cvs=cross_val_score(xgb6,x_train,y_train,scoring='roc_auc',verbose=10,n_jobs=1,cv=10)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\menta\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV]  ................................................................
[18:11:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV] .................................... , score=0.838, total=   1.4s
[CV]  ................................................................
[18:11:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
C:\Users\menta\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV] .................................... , score=0.842, total=   1.5s
[CV]  ................................................................
[18:11:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.8s remaining:    0.0s
C:\Users\menta\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   11.9s remaining:    0.0s
C:\Users\menta\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV] .................................... , score=0.835, total=   1.4s
[CV]  ................................................................
[18:11:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   13.3s remaining:    0.0s
C:\Users\menta\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[CV] .................................... , score=0.835, total=   1.5s


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   14.8s finished


In [48]:
scores=[0.838,0.842,0.803,0.810,0.855,0.883,0.843,0.828,0.835,0.835]

In [49]:
np.mean(scores)

0.8371999999999999

In [50]:
np.std(scores)

0.02115561391215106